In [8]:
# prompt: load dataset from huggingface ai4privacy/pii-masking-300k

#!pip install datasets

from datasets import load_dataset

dataset = load_dataset("ai4privacy/pii-masking-300k")


In [9]:
# prompt: load dataset from huggingface ai4privacy/pii-masking-300k

dataset2 = load_dataset("ai4privacy/pii-masking-200k")


In [10]:
from datasets import concatenate_datasets

column_name = "source_text"

data1_train = dataset['train'][column_name]
data2_train = dataset2['train'][column_name]

data1_test = dataset['validation'][column_name]

data1 = data1_train + data1_test
data2 = data2_train


# Convert columns to sets to avoid duplicates and simplify comparison
set_data1 = set(data1)
set_data2 = set(data2)



In [11]:
#len(set_data1 - set_data2)
set_data1.intersection(set_data2)

set()

In [12]:
pip install datasketch


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: datasketch in c:\users\win10\appdata\local\pypoetry\cache\virtualenvs\presidio-evaluator-ifenqhla-py3.12\lib\site-packages (1.6.5)




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from tqdm.notebook import tqdm
from multiprocessing import Pool, cpu_count


In [14]:
#create a machine learning model to check similarity between two things, which can be made to be the
#two datasets.

"""from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load a compact model from Hugging Face under 50M parameters
model_name = "sentence-transformers/all-MiniLM-L6-v2"  # ~33M parameters
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to create embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation = True)

    outputs = model(**inputs)
    # Use mean pooling of the last hidden state as sentence embedding
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()

# Sample data for similarity comparison
text1 = "This is a sample sentence."
text2 = "This sentence is a sample."
text3 = "Completely different content."

# Generate embeddings
embedding1 = get_embedding(text1)
embedding2 = get_embedding(text2)
embedding3 = get_embedding(text3)

# Calculate cosine similarity
# Calculate cosine similarity
similarity_1_2 = cosine_similarity(embedding1, embedding2)
similarity_1_3 = cosine_similarity(embedding1, embedding3)

print("Similarity between text1 and text2:", similarity_1_2[0][0])  # Should be high if similar
print("Similarity between text1 and text3:", similarity_1_3[0][0])  # Should be low if different"""

'from transformers import AutoTokenizer, AutoModel\nimport torch\nimport numpy as np\nfrom sklearn.metrics.pairwise import cosine_similarity\n\n# Load a compact model from Hugging Face under 50M parameters\nmodel_name = "sentence-transformers/all-MiniLM-L6-v2"  # ~33M parameters\ntokenizer = AutoTokenizer.from_pretrained(model_name)\nmodel = AutoModel.from_pretrained(model_name)\n\n# Function to create embeddings\ndef get_embedding(text):\n    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation = True)\n\n    outputs = model(**inputs)\n    # Use mean pooling of the last hidden state as sentence embedding\n    embeddings = outputs.last_hidden_state.mean(dim=1)\n    return embeddings.detach().numpy()\n\n# Sample data for similarity comparison\ntext1 = "This is a sample sentence."\ntext2 = "This sentence is a sample."\ntext3 = "Completely different content."\n\n# Generate embeddings\nembedding1 = get_embedding(text1)\nembedding2 = get_embedding(text2)\nembedding3 = get_

In [16]:
# Install necessary libraries
#!pip install datasets datasketch tqdm

from datasets import load_dataset, concatenate_datasets
from datasketch import MinHash, MinHashLSH
from tqdm.notebook import tqdm


def load_pii_500k():
    """
    Loads and concatenates the 'source_text' column from 'ai4privacy/pii-masking-300k'
    and 'ai4privacy/pii-masking-200k' datasets, avoiding feature mismatches.

    Returns:
        tuple: The train and test splits of the combined datasets.
    """
    # Load datasets
    dataset1 = load_dataset("ai4privacy/pii-masking-300k")
    dataset2 = load_dataset("ai4privacy/pii-masking-200k")

    # Select only the 'source_text' column to avoid feature alignment issues
    dataset1_train = dataset1['train'].select_columns(['source_text'])
    dataset2_train = dataset2['train'].select_columns(['source_text'])
    dataset1_test = dataset1['validation'].select_columns(['source_text'])

    # Concatenate train and test splits
    combined_train = concatenate_datasets([dataset1_train, dataset2_train])
    combined_test = dataset1_test  # Assuming only dataset1 has validation

    return combined_train, combined_test

def preprocess_data(dataset, column_name):
    """
    Extracts and prepares data from a specified column in the dataset.

    Args:
        dataset (Dataset): The Hugging Face dataset.
        column_name (str): The column to extract data from.

    Returns:
        set: Set of unique rows from the specified column.
    """
    data = dataset[column_name]
    return set(data)  # Convert to a set to simplify comparison


def exact_match_check(set_data1, set_data2):
    """
    checks for exact matches between two sets of data.
    just looks for if there's overlap

    Returns:
        set: Set of rows that are common between set_data1 and set_data2.
    """
    return set_data1.intersection(set_data2)


#def approximate_similarity_check(set_data1, set_data2, threshold=0.8, num_perm=32):
    """
    use LSH to find approximate matches between two datasets.

    Args:
        set_data1 (set): Set of data from the first dataset.
        set_data2 (set): Set of data from the second dataset.
        threshold (float): Similarity threshold for LSH.
        num_perm (int): Number of hash permutations.

    Returns:
        list: List of rows in set_data2 that have approximate matches in set_data1.
    """
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

    # Helper function to create MinHash objects
    def get_minhash(data_row):
        m = MinHash(num_perm=num_perm)
        for token in data_row:
            m.update(token.encode('utf8'))
        return m

    # Insert set_data1 into LSH
    for i, row in tqdm(enumerate(set_data1), desc="Indexing set_data1"):
        minhash = get_minhash(row)
        lsh.insert(f"data1_{i}", minhash)

    # Query LSH with set_data2
    shared_rows = []
    for i, row in tqdm(enumerate(set_data2), desc="Querying set_data2"):
        minhash = get_minhash(row)
        result = lsh.query(minhash)
        if result:
            shared_rows.append(row)

    return shared_rows


def compare_datasets_pipeline():
    """
    Pipeline to load datasets, check exact and approximate similarities, and return results.

    Returns:
        dict: Dictionary containing exact matches and approximate matches.
    """
    # Load datasets and preprocess
    train_data, test_data = load_pii_500k()
    column_name = "source_text"  # The column to compare

    set_data1 = preprocess_data(train_data, column_name)
    set_data2 = preprocess_data(test_data, column_name)

    # Exact match check
    exact_matches = exact_match_check(set_data1, set_data2)
    print(f"Number of exact matches: {len(exact_matches)}")

    # Approximate similarity check with LSH
    #shared_rows = approximate_similarity_check(set_data1, set_data2)
    #print(f"Number of approximate matches: {len(shared_rows)}")

    return {
        "exact_matches": exact_matches,
        #"approximate_matches": shared_rows
    }


# Run the pipeline
results = compare_datasets_pipeline()
print("Exact Matches:", results["exact_matches"])
#print("Approximate Matches:", results["approximate_matches"])


Number of exact matches: 0
Exact Matches: set()
